In [1]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_list_module import *


In [2]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [3]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:2000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(2000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [4]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [5]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [6]:
print(test_query_generator())

[np.float64(71.91244888776002), np.float64(0.9053334481516068), np.float64(0.7056014058078738), np.float64(2.3368910272417205), np.float64(-0.39943443520361654), np.float64(0.9876524616515067), np.float64(0.8463829985279863), np.float64(0.5398113405516464), np.float64(0.9058775762236672), np.float64(0.9233095315259685), np.float64(0.7218405895479987), np.float64(111.44771396710507), np.float64(198512.4428409201), np.float64(1.5788063234606136)]


In [7]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [8]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_])
# test_tree.show_root()
with Timer():
    test_tree.build_tree(min_threshold_for_clustering=200)

cur node 0
optimal K 2
[]
all [[]]
[0, 1, 3] 

cur node 1
optimal K 6
[[np.float64(1.0137778960074872)]]
cur node 2
optimal K 6
[[np.float64(1.0137778960074872)]]
all [[], [[np.float64(1.0137778960074872)]]]
[0, 1, 3, 15] 

cur node 3
[[np.float64(0.3861209361318126), np.float64(0.5918871064718276), np.float64(1.6627004442624695), np.float64(0.5968458152195195), np.float64(0.2402381551777252), np.float64(1.1841564912028961), np.float64(1.5361554596153573), np.float64(1.7699974491749837), np.float64(1.07732911677578), np.float64(2.7152727443373736), np.float64(1.537725119228167)]]
cur node 4
[[np.float64(0.3861209361318126), np.float64(0.5918871064718276), np.float64(1.6627004442624695), np.float64(0.5968458152195195), np.float64(0.2402381551777252), np.float64(1.1841564912028961), np.float64(1.5361554596153573), np.float64(1.7699974491749837), np.float64(1.07732911677578), np.float64(2.7152727443373736), np.float64(1.537725119228167)], [np.float64(0.6787666560784847), np.float64(1.6959

In [9]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [10]:
t = 200000
while t>100:
    seaching = test_query_generator()
    with Timer():
        cur = test_tree.search(seaching)
    t=cur.id
    print(cur.id)

Elapsed time: 0.0005681999900843948 seconds
0


In [11]:
a = test_tree._get_closest_node_within_lv(seaching, cur)
print([n.id for n in a])
print(cur.down_lv)

will return a sorted list of leaf node
[0]
poped 0
[1, 2]
poped 1
poped 2
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
poped 3
cal True False
poped 4
cal True False
poped 5
cal True False
poped 6
cal True False
poped 7
cal True False
poped 8
cal True False
poped 9
poped 10
cal True False
poped 11
cal True False
poped 12
poped 13
cal True False
poped 14
cal True False
[15, 16, 17, 18]
poped 15
poped 16
cal True False
poped 17
cal True False
poped 18
cal True False
[19, 20]
poped 19
cal True False
poped 20
cal True False
[]
res [5, 14, 20, 19, 3, 18, 4, 8, 16, 17, 7, 11, 6, 10, 13]
[5, 14, 20, 19, 3, 18, 4, 8, 16, 17, 7, 11, 6, 10, 13]
[<graph_tree_list_module.Node object at 0x0000011D1D1E12B0>, <graph_tree_list_module.Node object at 0x0000011D1B5F6780>]


In [12]:
test_tree.print_tree(test_tree.nodes[0])

└── 0
    ├── 1
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   └── 8
    └── 2
        ├── 9
        │   ├── 15
        │   │   ├── 19
        │   │   └── 20
        │   └── 16
        ├── 10
        ├── 11
        ├── 12
        │   ├── 17
        │   └── 18
        ├── 13
        └── 14


In [13]:
print(test_tree.nodes[0].center)

[0.44036364 0.5291267  0.65660264 0.47645455 0.76121848 0.6405
 0.06197007 0.25581723 0.13625976 0.18137536 0.48564998 0.48213134
 0.18333419 0.728875  ]


In [14]:
res = test_tree.reflection_search(np.mean(normalized_data[:100], axis=0), node=test_tree.nodes[11], degree=2)

0
will return a sorted list of leaf node
[0]
poped 0
[1, 2]
poped 1
poped 2
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
poped 3
cal True False
poped 4
cal True False
poped 5
cal True False
poped 6
cal True False
poped 7
cal True False
poped 8
cal True False
poped 9
poped 10
cal True False
poped 11
cal True False
poped 12
poped 13
cal True False
poped 14
cal True False
[15, 16, 17, 18]
poped 15
poped 16
cal True False
poped 17
cal True False
poped 18
cal True False
[19, 20]
poped 19
cal True False
poped 20
cal True False
[]
res [19, 16, 18, 3, 17, 20, 8, 10, 4, 11, 7, 14, 5, 13, 6]
will return a one closest node within lv
[0]
poped 0
[1, 2]
poped 1
cal False True
poped 2
cal False True
res [2]


In [15]:
res

[[<graph_tree_list_module.Node at 0x11d1b5f6780>],
 array([0.45153535, 0.52339993, 0.68550112, 0.49472727, 0.76952637,
        0.561     , 0.06974494, 0.27134241, 0.13702421, 0.19445554,
        0.47458816, 0.49446303, 0.19104194, 0.73775   ]),
 [np.int64(934),
  np.int64(461),
  np.int64(767),
  np.int64(1807),
  np.int64(1065),
  np.int64(915),
  np.int64(107),
  np.int64(1006),
  np.int64(1318),
  np.int64(803)]]

In [16]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  181
hitted right result: 12
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(2): 20, np.int32(4): 20, np.int32(6): 14, np.int32(1): 12, np.int32(8): 9, np.int32(5): 7, np.int32(0): 6, np.int32(3): 6, np.int32(7): 5, np.int32(9): 1})
belongs to group:  2 

-- mine result100
search number:  2000
hitted right result: 100
-- kmeans result
Counter({np.int32(4): 410, np.int32(2): 271, np.int32(6): 261, np.int32(1): 249, np.int32(8): 224, np.int32(0): 158, np.int32(5): 145, np.int32(7): 115, np.int32(3): 90, np.int32(9): 77})
Counter({np.int32(4): 22, np.int32(6): 11, np.int32(1): 11, np.int32(2): 11, np.int32(5): 10, np.int32(0): 10, np.int32(7): 8, np.int32(9): 7, np.int32(8): 7, np.int32(3): 3})
belongs to group:  4 

-- mine result200
search number:  2000
hitted right result: 100
-- kme

In [17]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[0.4695000904999095, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 0.4095001804998195, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07, 9.99998999939855e-07]
[0.46450013549986446, 0.40500020499979494, 0.37000020499979497, 0.41500020499979495, 0.40000020499979494, 0.43000020499979497, 0.3850002049997949, 0.41500020499979495, 0.42500020499979496, 0.4645001304998695]
0.8790082709917285
4.174001905998094
